In [ ]:
import pandas as pd
from pathlib import Path

pd.__version__

## Cleanup
- remove unused sheets
- remove irrelevant columns
- rename columns

In [ ]:
def collect_sheets(filepath, sheets_key, old_filename, new_fname):
    raw_excelfile = pd.read_excel(
        Path(filepath, old_filename), sheet_name=list(sheets_key.keys())
    )
    target_file = Path(filepath, new_fname)

    with pd.ExcelWriter(target_file, engine="openpyxl", mode="w") as writer:
        for sheet, new_sheet_name in sheets_key.items():
            raw_excelfile.get(sheet).to_excel(
                writer, sheet_name=new_sheet_name, index=False
            )


def process_columns(filepath, sheets_key, columns_key, old_filename):
    raw_excelfile = pd.ExcelFile(Path(filepath, old_filename))

    combined_df = pd.DataFrame()
    for sheet in sheets_key.values():
        df = pd.read_excel(raw_excelfile, sheet_name=sheet)
        df = df[columns_key.keys()]
        df = df.rename(columns=columns_key)
        df = process_rows(df)
        df['category'] = sheet
        combined_df = pd.concat([combined_df, df])

    combined_df.to_csv(f"./csv/combined.csv", index=False)


def process_rows(df):
    df['product_name'] = df['product_name'].str.title()
    df['color'] = df['color'].str.lower()
    df['style'] = df['style'].str.lower()
    df = df[df["price"] != "NFS"]

    return df


SHEETS_TO_USE = {
    "Tops": "tops",
    "Bottoms": "bottoms",
    "Dress-Up": "dressup",
    "Headwear": "headwear",
    "Accessories": "accessories",
    "Socks": "socks",
    "Shoes": "shoes",
    "Bags": "bags",
}
COLUMNS_TO_USE = {
    "Name": "product_name",
    "Buy": "price",
    "Color 1": "color",
    "Style 1": "style",
    "Filename": "filename",
    "Description": "description",  # generated
}

excel_filename = "Data Spreadsheet for Animal Crossing New Horizons.xlsx"
excel_path = Path(Path().cwd(), "scrape")

# collect_sheets(excel_path, SHEETS_TO_USE, excel_filename, "collected_sheets.xlsx")

gpt_excel_filename = "collected_with_desc.xlsx"
process_columns(excel_path, SHEETS_TO_USE, COLUMNS_TO_USE, gpt_excel_filename)